In [ ]:
pip install lime

In [ ]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import os

# Define the path to the MRI data and the model
mri_data_path = 'C:/Users/KIIT/Downloads/Brain-Tumor-Detection-master/Brain-Tumor-Detection-master/augmented data/no'
model_path = 'C:/Users/KIIT/Downloads/Brain-Tumor-Detection-master/Brain-Tumor-Detection-master/braintumordetector.h5'

# Load the images from the directory
def load_images_from_folder(folder, target_size):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize image to the target size
            images.append(img)
    return images

# The model's expected input size
input_size = (240, 240)

mri_images = load_images_from_folder(mri_data_path, input_size)

# Assuming labels are available in a separate file or derived from folder structure
# For this example, I'm assigning dummy labels (0) to all images
# Replace this with actual label loading mechanism
labels = [0] * len(mri_images)

print(f"Number of images: {len(mri_images)}, Image shape: {mri_images[0].shape if mri_images else 'No images'}")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"The model file {model_path} does not exist.")

inet_model = load_model(model_path)

explainer = lime_image.LimeImageExplainer()

# Select an image to explain
i = 0  # Change index as needed
image = mri_images[i]

print("Selected image shape:", image.shape)

image = np.expand_dims(image, axis=0)  
image = preprocess_input(image)

predictions = inet_model.predict(image)

explanation = explainer.explain_instance(image[0], inet_model.predict, top_labels=5, hide_color=0, num_samples=1000)

temp, mask = explanation.get_image_and_mask(
    label=predictions.argmax(axis=1)[0], 
    positive_only=True, 
    num_features=10, 
    hide_rest=False
)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(mri_images[i], cv2.COLOR_BGR2RGB))  
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(mark_boundaries(temp / 255.0, mask))  
plt.title('LIME Explanation')
plt.show()


In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

In [ ]:
#Select the same class explained on the figures above.
ind =  explanation.top_labels[0]

#Map each explanation weight to the corresponding superpixel
dict_heatmap = dict(explanation.local_exp[ind])
heatmap = np.vectorize(dict_heatmap.get)(explanation.segments)

#Plot. The visualization makes more sense if a symmetrical colorbar is used.
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()